# Training Yolov11 with RAAD
Training Yolo11n Model wtih Metric RAAD

In this Notebook a Yolo Model will be Trained on the Metric RAAD

In [1]:
!pip install -U ultralytics wandb
!pip install dotenv
!pip install shapely

In [24]:
import os
import numpy as np
import torch
import torch.nn as nn
import cv2
import wandb
from ultralytics import YOLO
from ultralytics.utils.metrics import Metric
from ultralytics.engine.trainer import BaseTrainer
from tqdm.notebook import tqdm
from shapely.geometry import box, MultiPolygon
import pandas as pd
from typing import List, Tuple, Dict, Optional
import logging

In [12]:
# Enable W&B logging for Ultralytics
!yolo settings wandb=True

JSONDict("/home/jovyan/.config/Ultralytics/settings.json"):
{
  "settings_version": "0.0.6",
  "datasets_dir": "/home/jovyan/DSPRO2/M-AI-ZE-Maize-diseases-detection/notebooks/datasets",
  "weights_dir": "weights",
  "runs_dir": "runs",
  "uuid": "8a115bbf5049f0fe55cf2ccd8be54ca8bfded6b963fd272724a959bb525556d2",
  "sync": true,
  "api_key": "",
  "openai_api_key": "",
  "clearml": true,
  "comet": true,
  "dvc": true,
  "hub": true,
  "mlflow": true,
  "neptune": true,
  "raytune": true,
  "tensorboard": false,
  "wandb": true,
  "vscode_msg": true,
  "openvino_msg": true
}
💡 Learn more about Ultralytics Settings at https://docs.ultralytics.com/quickstart/#ultralytics-settings


In [13]:
# Load the .env file
load_dotenv()

# Get and print the WANDB_API_KEY
wandb_api_key = os.getenv("WANDB_API_KEY")
print(f"WANDB_API_KEY: [{wandb_api_key[:4]}...]")

WANDB_API_KEY: [69ca...]


In [14]:
wandb.login(key=wandb_api_key)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/jovyan/.netrc


True

## Get Data

In [15]:
base_path = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits"
splits = ["SID01", "SID02", "SID03"]
test_csv_path = base_path + "/SID01/labels/test/bboxes_test.csv"

In [16]:
for split in splits:
    dataset_path = os.path.join(base_path, split)
    yaml_path = os.path.join(dataset_path, "data.yaml")
    
    print(f"Check Split: {split}")
    print(f"  Dataset Path: {dataset_path} - Existiert: {os.path.exists(dataset_path)}")
    print(f"  YAML Path: {yaml_path} - Existiert: {os.path.exists(yaml_path)}")

    if os.path.exists(yaml_path):
        with open(yaml_path, 'r') as f:
            data_yaml = yaml.safe_load(f)
            print(f"  YAML Inhalt: {data_yaml.keys()}")
            print(f"  Train: {data_yaml.get('train')}")
            print(f"  Val: {data_yaml.get('val')}")
            print(f"  Test: {data_yaml.get('test')}")
            print(f"  NC: {data_yaml.get('nc')}")
            print(f"  Names: {data_yaml.get('names')}")
    print()

Check Split: SID01
  Dataset Path: /exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01 - Existiert: True
  YAML Path: /exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/data.yaml - Existiert: True
  YAML Inhalt: dict_keys(['path', 'train', 'val', 'test', 'names'])
  Train: images/train
  Val: images/val
  Test: images/test
  NC: None
  Names: {0: 'lesion'}

Check Split: SID02
  Dataset Path: /exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID02 - Existiert: True
  YAML Path: /exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID02/data.yaml - Existiert: False

Check Split: SID03
  Dataset Path: /exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID03 - Existiert: True
  YAML Path: /exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID03/data.yaml - Existiert: False



## RAAD Implementation

In [17]:
def calculate_raad(pred_boxes, true_boxes, img_width=640, img_height=640, epsilon=1e-6, normalize=True):
    """
    Calculate RAAD (Relative Affected Area Difference) metric
    
    Args:
        pred_boxes: List of predicted boxes in format [x1, y1, x2, y2]
        true_boxes: List of ground truth boxes in format [x1, y1, x2, y2]
        img_width: Width of the image
        img_height: Height of the image
        epsilon: Small value to avoid division by zero
        normalize: Whether to normalize coordinates from 0-1 to image dimensions
        
    Returns:
        RAAD value (lower is better), Area of predictions, Area of ground truth
    """
    if len(pred_boxes) == 0 and len(true_boxes) == 0:
        return 0.0, 0.0, 0.0
    if len(pred_boxes) == 0:
        return 1.0, 0.0, sum([(box[2]-box[0])*(box[3]-box[1]) for box in true_boxes])
    if len(true_boxes) == 0:
        return 1.0, sum([(box[2]-box[0])*(box[3]-box[1]) for box in pred_boxes]), 0.0
    
    if normalize:
        pred_boxes = [[box[0]*img_width, box[1]*img_height, 
                      box[2]*img_width, box[3]*img_height] for box in pred_boxes]
        true_boxes = [[box[0]*img_width, box[1]*img_height, 
                      box[2]*img_width, box[3]*img_height] for box in true_boxes]
    
    pred_polygons = [box(b[0], b[1], b[2], b[3]) for b in pred_boxes]
    true_polygons = [box(b[0], b[1], b[2], b[3]) for b in true_boxes]
    
    if pred_polygons:
        pred_multipolygon = MultiPolygon(pred_polygons).buffer(0)
    else:
        pred_multipolygon = MultiPolygon([])
        
    if true_polygons:
        true_multipolygon = MultiPolygon(true_polygons).buffer(0)
    else:
        true_multipolygon = MultiPolygon([])
    
    pred_area = pred_multipolygon.area
    true_area = true_multipolygon.area
    
    area_diff = abs(pred_area - true_area)
    raad = area_diff / max(true_area, epsilon)
    
    return raad, pred_area, true_area

In [25]:
class RAADMetric:
    """RAAD (Relative Affected Area Difference) metric calculator"""
    
    def __init__(self):
        self.raad_values = []
        self.pred_areas = []
        self.true_areas = []
        
    def calculate_raad(self, pred_boxes, true_boxes, img_width=640, img_height=640, epsilon=1e-6, normalize=True):
        """Calculate RAAD metric between predicted and true boxes"""
        if len(pred_boxes) == 0 and len(true_boxes) == 0:
            return 0.0, 0.0, 0.0
        if len(pred_boxes) == 0:
            true_area = sum([(box[2]-box[0])*(box[3]-box[1]) for box in true_boxes])
            return 1.0, 0.0, true_area
        if len(true_boxes) == 0:
            pred_area = sum([(box[2]-box[0])*(box[3]-box[1]) for box in pred_boxes])
            return 1.0, pred_area, 0.0
        
        if normalize:
            pred_boxes = [[box[0]*img_width, box[1]*img_height, 
                          box[2]*img_width, box[3]*img_height] for box in pred_boxes]
            true_boxes = [[box[0]*img_width, box[1]*img_height, 
                          box[2]*img_width, box[3]*img_height] for box in true_boxes]
        
        try:
            pred_polygons = [box(b[0], b[1], b[2], b[3]) for b in pred_boxes if b[2] > b[0] and b[3] > b[1]]
            true_polygons = [box(b[0], b[1], b[2], b[3]) for b in true_boxes if b[2] > b[0] and b[3] > b[1]]
            
            pred_multipolygon = MultiPolygon(pred_polygons).buffer(0) if pred_polygons else MultiPolygon([])
            true_multipolygon = MultiPolygon(true_polygons).buffer(0) if true_polygons else MultiPolygon([])
            
            pred_area = pred_multipolygon.area
            true_area = true_multipolygon.area
            
            area_diff = abs(pred_area - true_area)
            raad = area_diff / max(true_area, epsilon)
            
            return raad, pred_area, true_area
        except Exception as e:
            print(f"Error in RAAD calculation: {e}")
            return 1.0, 0.0, 0.0
        
    def process_batch(self, detections, labels, img_width=640, img_height=640):
        """Process one batch of detections and labels"""
        batch_raad = []
        
        if detections is None or len(detections) == 0:
            detections = []
        if labels is None or len(labels) == 0:
            labels = []
            
        pred_boxes = []
        if len(detections) > 0:
            if torch.is_tensor(detections):
                detections = detections.cpu().numpy()
            for det in detections:
                if len(det) >= 4:
                    pred_boxes.append([det[0], det[1], det[2], det[3]])
        
        true_boxes = []
        if len(labels) > 0:
            if torch.is_tensor(labels):
                labels = labels.cpu().numpy()
            for label in labels:
                if len(label) >= 5:
                    # Labels format: [cls, cx, cy, w, h] normalized
                    cls, cx, cy, w, h = label[:5]
                    x1 = (cx - w/2) * img_width
                    y1 = (cy - h/2) * img_height
                    x2 = (cx + w/2) * img_width
                    y2 = (cy + h/2) * img_height
                    true_boxes.append([x1, y1, x2, y2])
        
        raad, pred_area, true_area = self.calculate_raad(
            pred_boxes, true_boxes, img_width, img_height, normalize=False
        )
        
        self.raad_values.append(raad)
        self.pred_areas.append(pred_area)
        self.true_areas.append(true_area)
        
        return raad
        
    def mean_raad(self):
        """Calculate mean RAAD over all processed batches"""
        if not self.raad_values:
            return 0.0
        return sum(self.raad_values) / len(self.raad_values)
    
    def reset(self):
        """Reset metrics for new evaluation"""
        self.raad_values = []
        self.pred_areas = []
        self.true_areas = []


## Custom Yolo Trainer


In [19]:
class CustomDetectionValidator(DetectionValidator):
    """Custom validator that calculates RAAD during validation"""
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.raad_metric = RAADMetric()
        
    def init_metrics(self, model):
        """Initialize metrics including RAAD"""
        super().init_metrics(model)
        self.raad_metric.reset()
        
    def postprocess(self, preds):
        """Postprocess predictions and calculate RAAD"""
        processed_preds = super().postprocess(preds)
        
        try:
            if hasattr(self, 'batch') and self.batch is not None:
                # Get current batch labels
                labels = self.batch.get('bboxes', None)
                
                if labels is not None and processed_preds is not None:
                    for i, pred in enumerate(processed_preds):
                        if i < len(labels) and labels[i] is not None:
                            # Convert predictions to boxes
                            if pred is not None and len(pred) > 0:
                                pred_boxes = pred[:, :4].cpu().numpy()  # x1, y1, x2, y2
                            else:
                                pred_boxes = []
                            
                            if len(labels[i]) > 0:
                                label_boxes = []
                                for label in labels[i]:
                                    if len(label) >= 5:
                                        cls, cx, cy, w, h = label[:5]
                                        x1 = (cx - w/2) * self.args.imgsz
                                        y1 = (cy - h/2) * self.args.imgsz
                                        x2 = (cx + w/2) * self.args.imgsz
                                        y2 = (cy + h/2) * self.args.imgsz
                                        label_boxes.append([x1, y1, x2, y2])
                            else:
                                label_boxes = []
                            
                            raad, _, _ = self.raad_metric.calculate_raad(
                                pred_boxes.tolist() if len(pred_boxes) > 0 else [],
                                label_boxes,
                                self.args.imgsz, self.args.imgsz,
                                normalize=False
                            )
                            
                            self.raad_metric.raad_values.append(raad)
                            
        except Exception as e:
            print(f"Error in RAAD calculation during validation: {e}")
            
        return processed_preds
    
    def get_stats(self):
        """Get validation statistics including RAAD"""
        stats = super().get_stats()
        
        raad_mean = self.raad_metric.mean_raad()
        stats = list(stats) if isinstance(stats, tuple) else [stats]
        stats.append(raad_mean)
        
        return stats

In [26]:
class RAADLoss(nn.Module):
    """RAAD-based loss function for YOLO training"""
    
    def __init__(self, weight=1.0, reduction='mean'):
        super().__init__()
        self.weight = weight
        self.reduction = reduction
        self.raad_metric = RAADMetric()
        
    def forward(self, pred_boxes, true_boxes, img_size=640):
        """
        Calculate RAAD loss
        
        Args:
            pred_boxes: Predicted boxes tensor [N, 4] in xyxy format
            true_boxes: Ground truth boxes tensor [M, 4] in xyxy format
            img_size: Image size for normalization
            
        Returns:
            RAAD loss tensor
        """
        if pred_boxes.numel() == 0 and true_boxes.numel() == 0:
            return torch.tensor(0.0, device=pred_boxes.device, requires_grad=True)
            

        pred_np = pred_boxes.detach().cpu().numpy() if pred_boxes.numel() > 0 else []
        true_np = true_boxes.detach().cpu().numpy() if true_boxes.numel() > 0 else []
        
        raad, _, _ = self.raad_metric.calculate_raad(
            pred_np.tolist() if len(pred_np) > 0 else [],
            true_np.tolist() if len(true_np) > 0 else [],
            img_size, img_size, normalize=False
        )
        
        raad_tensor = torch.tensor(raad, device=pred_boxes.device, requires_grad=True)
        
        return raad_tensor * self.weight


In [27]:
class CustomYOLOTrainer:
    """Custom YOLO trainer with RAAD metric integration"""
    
    def __init__(self, model_path="yolo11n.pt", raad_weight=0.1):
        self.model = YOLO(model_path)
        self.raad_metric = RAADMetric()
        self.raad_loss = RAADLoss(weight=raad_weight)
        self.raad_weight = raad_weight
        
    def train_with_raad(self, data_yaml, epochs=10, batch=16, lr=0.01, project="maize_detection", name="raad_training"):
        """Train YOLO model with RAAD metric tracking"""
        
        wandb_run = wandb.init(
            project=project,
            name=name,
            config={
                "epochs": epochs,
                "batch_size": batch,
                "learning_rate": lr,
                "raad_weight": self.raad_weight,
                "model": str(type(self.model.model))
            },
            reinit=True
        )
        
        try:
            class RAADCallback:
                def __init__(self, raad_metric, wandb_run):
                    self.raad_metric = raad_metric
                    self.wandb_run = wandb_run
                    self.epoch_raad_values = []
                
                def on_train_epoch_start(self, trainer):
                    self.raad_metric.reset()
                    self.epoch_raad_values = []
                
                def on_val_start(self, validator):
                    self.raad_metric.reset()
                
                def on_val_batch_end(self, validator):
                    # For now, we'll skip per-batch RAAD calculation during training
                    # and rely on post-training evaluation
                    pass
                
                def on_val_end(self, validator):
                    if self.epoch_raad_values:
                        avg_raad = sum(self.epoch_raad_values) / len(self.epoch_raad_values)
                    else:
                        avg_raad = self.raad_metric.mean_raad()
                    
                    current_epoch = getattr(validator, 'epoch', 0)
                    
                    if self.wandb_run:
                        self.wandb_run.log({
                            "val/raad": avg_raad,
                            "epoch": current_epoch
                        })
                    
                    print(f"Epoch {current_epoch} - Validation RAAD: {avg_raad:.4f}")
            
            callback = RAADCallback(self.raad_metric, wandb_run)
            
            # Add callbacks to model
            self.model.add_callback("on_train_epoch_start", callback.on_train_epoch_start)
            self.model.add_callback("on_val_start", callback.on_val_start)
            self.model.add_callback("on_val_batch_end", callback.on_val_batch_end)
            self.model.add_callback("on_val_end", callback.on_val_end)
            
            results = self.model.train(
                data=data_yaml,
                epochs=epochs,
                imgsz=640,
                lr0=lr,
                batch=batch,
                name=name,
                project=project,
                exist_ok=True,
                verbose=True
            )
            
            return results
            
        finally:
            if wandb_run:
                wandb_run.finish()

## Training with RAAD

In [28]:
def train_model_with_raad(model_type="yolo11n.pt", split="SID01", epochs=10, batch=16, lr=0.01, raad_weight=0.1):
    """
    Enhanced training function with RAAD metric integration
    
    Args:
        model_type: YOLO model type/path
        split: Data split name
        epochs: Number of training epochs
        batch: Batch size
        lr: Learning rate
        raad_weight: Weight for RAAD loss component
        
    Returns:
        Path to trained model weights
    """
    base_path = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits"
    dataset_yaml = f"{base_path}/{split}/data.yaml"
    
    model_name = model_type.split('.')[0]
    run_name = f"{model_name}_{split}_raad_w{raad_weight}_e{epochs}"
    
    try:
        trainer = CustomYOLOTrainer(model_type, raad_weight=raad_weight)
        
        results = trainer.train_with_raad(
            data_yaml=dataset_yaml,
            epochs=epochs,
            batch=batch,
            lr=lr,
            project="maize_detection_raad",
            name=run_name
        )
        
        potential_paths = [
            f"runs/detect/{run_name}/weights/best.pt",
            f"maize_detection_raad/{run_name}/weights/best.pt",
            f"runs/train/{run_name}/weights/best.pt",
            f"maize_detection_raad/{run_name}/weights/last.pt"
        ]
        
        weights_path = None
        for path in potential_paths:
            if os.path.exists(path):
                weights_path = path
                print(f"Found trained model at: {weights_path}")
                break
        
        if weights_path is None:
            print("Warning: Could not find trained weights")
            return model_type
            
        return weights_path
        
    except Exception as e:
        print(f"Error during RAAD training: {e}")
        import traceback
        traceback.print_exc()
        return model_type

## Evaluation

In [22]:
def load_bounding_boxes_from_csv(csv_path):
    """
    Load bounding boxes from a single CSV file.
    
    Args:
        csv_path (str): Path to the CSV file.
        
    Returns:
        dict: Dictionary where keys are image names and values are lists of bounding boxes.
    """
    df = pd.read_csv(csv_path, header=None, skiprows=1)
    bounding_boxes = {}
    for _, row in df.iterrows():
        img_name = row[0]  # Bildname
        x1, y1, x2, y2 = map(int, [row[1], row[2], row[3], row[4]])
        if img_name not in bounding_boxes:
            bounding_boxes[img_name] = []
        bounding_boxes[img_name].append((x1, y1, x2, y2))
    print(f"Loaded bounding boxes for {len(bounding_boxes)} images.")
    return bounding_boxes

def suppress_yolo_logging():
    """Suppress the YOLO logging temporarily."""
    logger = logging.getLogger("ultralytics")
    original_level = logger.level
    logger.setLevel(logging.ERROR) 
    return logger, original_level

def restore_yolo_logging(logger, original_level):
    """Restore the YOLO logging to its original level."""
    logger.setLevel(original_level)

def evaluate_test_raad(model_path, bounding_boxes, split="SID01"):
    """
    Evaluate a trained model on test data with RAAD metric.
    
    Args:
        model_path (str): Path to the YOLO model file.
        bounding_boxes (dict): Dictionary with image names as keys and box lists as values.
        split (str): Data split name.
        
    Returns:
        tuple: (DataFrame with results, average RAAD value)
    """
    model_name = os.path.basename(model_path).split('.')[0]
    
    run = wandb.init(project="maize_detection_test", 
                    name=f"test_evaluation_{model_name}_{split}",
                    config={"model_path": model_path, "split": split})
    
    try:
        base_path = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits"
        test_images_dir = f"{base_path}/{split}/images/test"
        
        model = YOLO(model_path)
        raad_calculator = RAADMetric()  # Use our RAADMetric class
        
        results = {
            'image': [],
            'raad': [],
            'pred_area': [],
            'true_area': [],
            'prediction_count': [],
            'truth_count': []
        }
        
        test_images = [f for f in os.listdir(test_images_dir) if f.endswith(('.jpg', '.png'))]
        
        for img_file in tqdm(test_images, desc="Evaluating test images"):
            img_path = os.path.join(test_images_dir, img_file)
            
            if img_file not in bounding_boxes:
                print(f"No ground truth for image: {img_file}")
                continue
            
            img = cv2.imread(img_path)
            img_height, img_width = img.shape[:2]
            
            logger, original_level = suppress_yolo_logging()
            try:
                predictions = model.predict(img_path, save=False)
            finally:
                restore_yolo_logging(logger, original_level)
            
            pred_boxes = [
                [int(box[0]), int(box[1]), int(box[2]), int(box[3])]
                for r in predictions for box in r.boxes.xyxy.cpu().numpy()
            ]
            
            true_boxes = list(bounding_boxes[img_file])  # Convert tuples to list
            
            raad, pred_area, true_area = raad_calculator.calculate_raad(
                pred_boxes, true_boxes, img_width, img_height, normalize=False
            )
            
            results['image'].append(img_file)
            results['raad'].append(raad)
            results['pred_area'].append(pred_area)
            results['true_area'].append(true_area)
            results['prediction_count'].append(len(pred_boxes))
            results['truth_count'].append(len(true_boxes))
        
        results_df = pd.DataFrame(results)
        avg_raad = results_df['raad'].mean()
        print(f"Average RAAD on test set: {avg_raad:.4f}")
        run.log({"test/avg_raad": avg_raad})
        run.finish()
        
        return results_df, avg_raad
    
    except Exception as e:
        print(f"Error during RAAD evaluation: {e}")
        import traceback
        traceback.print_exc()
        run.finish()
        return None, None

In [29]:
def evaluate_raad_training_effectiveness(original_model, raad_trained_model, test_data_path, split="SID01"):
    """
    Compare RAAD performance between original and RAAD-trained models
    
    Args:
        original_model: Path to original model
        raad_trained_model: Path to RAAD-trained model
        test_data_path: Path to test CSV file
        split: Data split name
        
    Returns:
        Comparison results dictionary
    """
    print("Evaluating RAAD training effectiveness...")
    
    bounding_boxes = load_bounding_boxes_from_csv(test_data_path)
    
    print("Evaluating original model...")
    orig_results, orig_raad = evaluate_test_raad(original_model, bounding_boxes, split)
    
    print("Evaluating RAAD-trained model...")
    raad_results, raad_trained_raad = evaluate_test_raad(raad_trained_model, bounding_boxes, split)
    
    if orig_raad is not None and raad_trained_raad is not None:
        improvement = ((orig_raad - raad_trained_raad) / orig_raad) * 100
        
        results = {
            "original_raad": orig_raad,
            "raad_trained_raad": raad_trained_raad,
            "improvement_percent": improvement,
            "original_results": orig_results,
            "raad_trained_results": raad_results
        }
        
        print(f"Original model RAAD: {orig_raad:.4f}")
        print(f"RAAD-trained model RAAD: {raad_trained_raad:.4f}")
        print(f"Improvement: {improvement:.2f}%")
        
        wandb_run = wandb.init(
            project="raad_training_comparison",
            name=f"comparison_{split}",
            config={"split": split}
        )
        
        wandb_run.log({
            "original_raad": orig_raad,
            "raad_trained_raad": raad_trained_raad,
            "improvement_percent": improvement
        })
        
        wandb_run.finish()
        
        return results
    else:
        print("Error: Could not evaluate one or both models")
        return None

## MAIN

In [30]:
if __name__ == "__main__":
    raad_trained_path = train_model_with_raad(
        model_type="yolo11n.pt",
        split="SID01",
        epochs=5,
        batch=16,
        lr=0.01,
        raad_weight=0.1
    )
    
    print(f"RAAD-trained model saved at: {raad_trained_path}")
    
    base_path = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits"
    test_csv_path = f"{base_path}/SID01/labels/test/bboxes_test.csv"
    
    comparison = evaluate_raad_training_effectiveness(
        original_model="yolo11n.pt",
        raad_trained_model=raad_trained_path,
        test_data_path=test_csv_path,
        split="SID01"
    )

wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


Ultralytics 8.3.142 🚀 Python-3.12.8 torch-2.6.0+cu124 CUDA:0 (NVIDIA A16, 14891MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11n_SID01_raad_w0.1_e5, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patienc

train: Scanning /exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/labels/train.cache... 10858 images, 0 backgrounds, 0 corrupt: 100%|██████████| 10858/10858 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1728.6±782.3 MB/s, size: 56.1 KB)


val: Scanning /exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/labels/val.cache... 1357 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1357/1357 [00:00<?, ?it/s]


...

Evaluating test images:   0%|          | 0/1358 [00:00<?, ?it/s]

Average RAAD on test set: 0.9025


test/avg_raad,▁
test/avg_raad,0.90251


Original model RAAD: 4.0873
RAAD-trained model RAAD: 0.9025
Improvement: 77.92%


improvement_percent,▁
original_raad,▁
raad_trained_raad,▁
improvement_percent,77.91917
original_raad,4.08729
raad_trained_raad,0.90251
